# Load Data

In [7]:
# text = train['comment_text'][index]
# text

In [8]:
# # import pandas as pd
# # text = pd.DataFrame(train['comment_text'][:10])

# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# train['comment_text'].map(tokenizer.encode_plus(,
#                      None,
#                      add_special_tokens =True,
#                      max_length = 64,
#                      pad_to_max_length = True,
#                      return_token_type_ids = True)
# input_

In [9]:
import numpy as np
import pandas as pd

# read the train dataset
train = pd.read_csv(r'/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
# Look at dataset
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
## Comment_text is our feature , which we will tokenize using bert toknizer 
## Toxic is our label

## Load test and validation dataset
test = pd.read_csv(r'/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
validation = pd.read_csv(r'/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')

In [11]:
## Run code on small sample
train = train[:1000]
validation = validation[:1000]
test = test[:1000]

## Set up TPU

In [12]:
# # Detect hardware, return appropriate distribution strategy
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is
#     # set: this is always the case on Kaggle.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
#     strategy = tf.distribute.get_strategy()

# print("REPLICAS: ", strategy.num_replicas_in_sync)

# Preprocessing

Process individual sentences for input to BERT using the tokenizer, and then prepare the entire dataset. The same code will process the other training data files, as well as the validation and test data.

We need to clean the comment text feature , we will use NLTK library for cleaning the text , then use BERT MODEL TO TOKENIZE

In [13]:
# Make the text into lowercase 
train['comment_text'] = train['comment_text'].apply(lambda x : str(x).lower())



In [14]:
## Check if it is lowercase
train['comment_text'][:4]

0    explanation\nwhy the edits made under my usern...
1    d'aww! he matches this background colour i'm s...
2    hey man, i'm really not trying to edit war. it...
3    "\nmore\ni can't make any real suggestions on ...
Name: comment_text, dtype: object

In [15]:
## Remove stopwords
import nltk
from nltk.corpus import stopwords

stopword = stopwords.words('english')

## define a for loop to get function
def remove_stopwords(txt):
    words = txt.split(' ')
    filtered_word = []
    for word in words:
        if word not in stopword:
            filtered_word.append(word)
    return ' '.join(filtered_word)

        
        

In [16]:
print(f"Text in the first row is {train['comment_text'][:1]} \n")
word = train['comment_text'][:1].apply(remove_stopwords)
print(f"Text in the first row after removing stop word is {word}")
      

Text in the first row is 0    explanation\nwhy the edits made under my usern...
Name: comment_text, dtype: object 

Text in the first row after removing stop word is 0    explanation\nwhy edits made username hardcore ...
Name: comment_text, dtype: object


In [17]:
## Apply it in the dataset
train['comment_text'] = train['comment_text'].apply(remove_stopwords)

In [18]:
import re,string
## Remove special Characters
## define a function to clean the text
def clean_text(txt):
    ## make it lowercase
    txt = str(txt).lower()
    ## remove any sequence of characters in square brackets
    ## remove special characters .*?
    ## remove brackets hence use backlash \[ , \]
    ## re.sub(pattern,replacement,string)
    txt = re.sub('\[.*?\]','',txt)
    ## remove links
    txt =re.sub('https?://\S+|www\.\S+','',txt)
    ## remove HTML tags
    txt = re.sub('<.*?>+','',txt)
    ## remove punctuations
    txt = re.sub('[%s]' % re.escape(string.punctuation),'',txt)
    ## remove newline characters
    txt = re.sub('\n','',txt)
    ## remove words containing numbers
    txt = re.sub('\w*\d\w*','',txt)
    return txt

In [19]:
train['comment_text'] = train['comment_text'].apply(clean_text)
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanationwhy edits made username hardcore me...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matches background colour im seemingly st...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really trying edit war guy constant...,0,0,0,0,0,0
3,0001b41b1c6bb37e,morei cant make real suggestions improvement ...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir hero chance remember page thats on,0,0,0,0,0,0


In [20]:
## clean the test and validation data aslo
test['lang'].unique()

array(['tr', 'ru', 'it', 'fr', 'pt', 'es'], dtype=object)

In [21]:
## we need to get the stopwords for all five languagee
#append all the other languages stopwords

tuk_stopwords = stopwords.words('turkish')
rus_stopwords = stopwords.words('russian')
ital_stopwords = stopwords.words('italian')
french_stopwords = stopwords.words('french')
portugese_stopwords = stopwords.words('portuguese')
spanish_stopwords = stopwords.words('spanish')

test_stopwords = tuk_stopwords + rus_stopwords + ital_stopwords + french_stopwords + portugese_stopwords + spanish_stopwords

## define a funct to remove stopwords from test data
def test_stopword(txt):
    words = txt.split(' ')
    filtered_word = []
    for word in words:
        if word not in test_stopwords:
            filtered_word.append(word)
    return ' '.join(filtered_word)

## remove stopwords from test data 
test['content'] = test['content'].apply(test_stopword)
test['content'] = test['content'].apply(clean_text)

In [22]:
test.head()

,id,content,lang
0,0,doctor who adlı viki başlığına doctor olarak ...,tr
1,1,вполне возможно пока вижу необходимости выдел...,ru
2,2,quindi conservativi preferiscono cancellar...,it
3,3,malesef gerçekleştirilmedi ancak şöyle bir var...,tr
4,4,resimseldabagcanjpg resminde kaynak sorunu ...,tr


In [23]:
validation['lang'].unique()

array(['es', 'it', 'tr'], dtype=object)

In [24]:
validation.head()

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0


In [25]:
## validation stopwords
validation_stopword = tuk_stopwords + ital_stopwords + spanish_stopwords

def valid_stop(txt):
    words = txt.split(' ')
    filtered_word = []
    for word in words:
        if word not in validation_stopword:
            filtered_word.append(word)
    return ' '.join(filtered_word)

validation['comment_text'] = validation['comment_text'].apply(valid_stop)
validation['comment_text'] = validation['comment_text'].apply(clean_text)

validation.head()

,id,comment_text,lang,toxic
0,0,este usuario siquiera llega rango hereje ...,es,0
1,1,il testo voce pare esser scopiazzato direttame...,it,0
2,2,vale sólo expongo pasado todo tiempo pasado me...,es,1
3,3,bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,belçika nın şehirlerinin yanında ilçe beldeler...,tr,0


In [26]:
## 1.Tokenization
import torch 
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [27]:
## encode the text
train['comment_text'] = train['comment_text'].apply(lambda x : tokenizer.encode(x,add_special_tokens = True))

In [28]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"[101, 88840, 10874, 19275, 70971, 10107, 11019...",0,0,0,0,0,0
1,000103f0d9cfb60f,"[101, 10143, 10874, 10874, 18258, 25903, 43361...",0,0,0,0,0,0
2,000113f07ec002fd,"[101, 10261, 10157, 10817, 10211, 30181, 32862...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"[101, 10798, 10116, 10944, 10123, 13086, 13486...",0,0,0,0,0,0
4,0001d958c54c6e35,"[101, 13028, 52523, 51670, 27893, 93161, 15975...",0,0,0,0,0,0


In [29]:
## take only two cilumns for new df, comment_text, toxic
new_train = train[['comment_text','toxic']]
new_train.head(2)

,comment_text,toxic
0,"[101, 88840, 10874, 19275, 70971, 10107, 11019...",0
1,"[101, 10143, 10874, 10874, 18258, 25903, 43361...",0


In [46]:
## 2. padding
## get the maximum length 
max_len = 0
for i in new_train['comment_text'].values:
    if len(i) > max_len:
        max_len = len(i)

In [47]:
# new_train['comment_text'].iloc[0]

In [48]:
max_len = 512
## now pad the tokens which are less than the max_len
padded_rows = []

for i in new_train['comment_text'].values:
        pad = [0] * (max_len - len(i))
        padded_rows.append(i + pad)
        

In [49]:
new_train['comment_text'] = padded_rows

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
## check first row for padding
new_train['comment_text'][1][-10:]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [51]:
## 3. Masking
## we need to tell bert to ignore(mask) the padding we have added
## when processing the input

attention_mask = np.where(new_train['comment_text'] != 0,1,0)
attention_mask.shape

(1000,)

In [52]:
## create tensor for model inputs
input_ids = torch.tensor(new_train['comment_text'])
attention_mask = torch.tensor(attention_mask)



In [53]:
import transformers
## feed into the model to get predictions
model = transformers.DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')


In [54]:
## Model 1 
## get predictions without training
# with ctrategy.scope():
with torch.no_grad():
    last_hidden_states = model(input_ids,attention_mask = attention_mask)

RuntimeError: index out of range: Tried to access index 512 out of table with 511 rows. at /opt/conda/conda-bld/pytorch_1579022034529/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

From here pass the hidden state througha linear function , then sigmoid function to get the final predicted class

In [ ]:
## BERT does sentence classification it adds token called [CLS]

BERT does sentence classification it adds token called [CLS] (for classification) at begining of every sentence, The output corresponding to that token can be related as an embedding of rntire sentence

From here we can use logistic regression using the [CLS] embedding layer as features and labels of our dataset, 
This is a way without fine tuninng the bert model 
we will do this approach later , for now lets move to pytorch architechture to fine tune your bert model  accrdng to our dataset and then ask it to classify


## APPROACH 2  BERT TOKENIZER
**Using pytorch transformer library**

In [ ]:
## import Libraries
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer,AutoTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np
from torch import cuda

In [ ]:
## Setup device
## clean Dataset
## Create class for dataset
## define training and validation parameters
## define class modeule for model
## define loss function
## define optimizer
## define evaluate function
## Run training loop
## get the predictions output and evaluation score
## create a submission file

In [ ]:
## Setup device
# device = "cuda" if torch.cuda.is_available() else "cpu"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
## take 500 rows to run the code
# train = train[:500]

In [ ]:
import torch
print("Is CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)
print(device)


In [ ]:
## define key cariables 
MAX_LEN = max(train['comment_text'].apply(len))
BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 0.001

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
                                         

In [ ]:
## Class for creating dataset

class multilingual_dataset(Dataset):
    def __init__(self, dataframe,tokenizer,max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __getitem__(self,index):
        text = str(self.data.comment_text[index])
        inputs = tokenizer.encode_plus(text = text,
                             text_pair = None,
                                 add_special_tokens = True,
                                 max_length = self.max_len,
                                 pad_to_max_length = True,
                                 return_token_type_ids = True,
                                 return_attention_mask = True,
                                       truncation_strategy = 'longest_first'
                                )
        idx = inputs['input_ids']
        mask = inputs["attention_mask"]
        return   {
            'ids': torch.tensor(idx, dtype = torch.long), # 64 bit integer
            'mask' : torch.tensor(mask, dtype = torch.long), # 64 bit integer
            'targets' : torch.tensor(self.data.toxic[index], dtype = torch.float32)
            
        }
    def __len__(self):
        return len(self.data)
        

In [ ]:
MAX_LEN

In [ ]:
training_set = multilingual_dataset(train,tokenizer,512)
validation_set = multilingual_dataset(validation,tokenizer,512)


In [ ]:
## load data into dataloader
from torch.utils.data import DataLoader
train_data = DataLoader(dataset = training_set,
                       batch_size=10,
                       num_workers = 2)
valid_data = DataLoader(dataset = validation_set,
                       batch_size=10,
                       num_workers = 2)

In [ ]:
## constructor


class multilingual_bert_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = transformers.DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
        self.drop = nn.Dropout(0.2)
        self.linear = nn.Linear(768,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,ids,mask):
        model_output = self.bert(ids,mask)
        hidden_state = model_output[0]
        first_token_of_every_sentence = hidden_state[:,0,:] #all sentences, CLS position, all hidden unit outputs
        output = self.drop(first_token_of_every_sentence)
        linear_output = self.linear(output)
        final_output = self.sigmoid(linear_output)
        return final_output

In [ ]:
# instantiate model
model = multilingual_bert_model()
## put it on device
model.to(device)
# define loss function
loss_fn = nn.BCELoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(),
                            lr = 0.01)

# define evaluation
from sklearn.metrics import roc_auc_score
def roc_score(y_train,y_pred):
    ## convert tensors to numpy
    y_train = y_train.detach().cpu().to_numpy()
    y_pred = y_pred.detach().cpu().to_numpy()
    roc_score = roc_auc_score(y_train,y_pred)
    return roc_score




In [ ]:
# for i,j in enumerate(train_data,0):
#     print(j)

In [ ]:
epochs = 20


for epoch in range(epochs):
    #train
    model.train()
    for _, data in enumerate(train_data):
        ids = data['ids'].to(device,dtype = torch.long)
        mask = data['mask'].to(device,dtype = torch.long)
        y_train = data['targets'].to(device,dtype = torch.float)
    # forward pas
    y_pred = model(ids,mask).squeeze()
    #calculate the loss
    loss = loss_fn(y_pred,y_train)
    #optimizer zero grad
    optimizer.zero_grad()
    #loss.backward
    loss.backward()
    # optimizer step
    optimizer.step()
    
    ##testing
    model.eval()
    with torch.no_grad():
        ## forwards pass
        for _,Data in enumerate(valid_data):
            idx = Data['ids'].to(device,dtype=torch.long)
            Mask = Data['mask'].to(device,dtype=torch.long)
            eval_y = Data['targets'].to(device,dtype=torch.float)
            
        eval_pred = model(ids = idx,mask = Mask)
        # calculate loss
        eval_loss = loss_fn(eval_pred,eval_y)
    
    #print whats happening
    if epoch % 10 ==0:
        print(f"Epoch: {epoch} | train_loss {loss} | eval_loss {eval_loss}")
    
    

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


In [ ]:
 
x = torch.tensor([1.0, 2.0]).cuda()
print(x)


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# Ensure your data tensors are also sent to the device
model